# Klasifikacija teksta

U ovoj svesci proći ćemo ponovo kroz primer koji smo uveli u priči o konvolutivnim neuronskim mrežama. Bavićemo se klasifikacijom IMDB filmskih pregleda na pozitivne i negativne, ali ovoga puta uz korišćenje rekurentnih neuronskih mreža. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, GRU, Flatten
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

In [ ]:
import pandas as pd
import re

Ovaj paket ćemo iskoristiti za praćenje vremena traniranja.

In [4]:
from time import perf_counter

## Priprema skupa podataka

Podsetimo se da prilikom učitavanja skupova za učenje i testiranje treba naglasiti sa kojom veličinom vokabulara želimo da radimo. Vokabular je uređen opadajuće po frekvenciji pojavljivanja reči, a navedeni broj predstavlja broj reči sa početka vokabulara. Ovde se zbog ograničenja zadržavamo na prvih 10000 reči. 

In [5]:
max_features = 5000

In [8]:
(X_train, y_train), (X_test, y_test)= imdb.load_data(num_words=max_features)

In [9]:
print('Skup za treniranje: ', X_train.shape)
print('Skup za testriranje: ', X_test.shape)

Skup za treniranje:  (25000,)
Skup za testriranje:  (25000,)


Podsetimo se i da je jedan ovako učitani pregled niz indeksa reči vokabulara. Nulama su predstavljene reči koje nisu pokrivene vokabularom.

In [10]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [11]:
len(X_train[0])

218

Očekuje se i da pre obrade sve preglede svedemo na iste dužine. Mi ćemo se opredeliti za dužinu od 500 reči. Pregledi koji sadrže manje reći biće dopunjeni nulama, a pregledi koji sadrže više reči biće skraćeni.

In [12]:
maxlen = 500

In [13]:
X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [14]:
print(X_train.shape)

(25000, 500)


In [15]:
print(X_test.shape)

(25000, 500)


Za dalje predstavljanje reči iskoristićemo ugnježdavanja u semantički prostor dimenzije `embedding_dimensionality` pomoću sloja `Embedding`.

Ciljna promenljiva sadrži samo vrednosti 0 i 1 koje redom opisuju negativne i pozitivne preglede. Broj pozitivnih i broj negativnih pregleda je isti.

In [16]:
np.unique(y_train)

array([0, 1])

In [17]:
negative_review = np.sum(y_train == 0)
positive_review = np.sum(y_train == 1)

In [18]:
print('Broj pozitivnih pregleda: ', positive_review)
print('Broj negitivnih pregleda: ', negative_review)

Broj pozitivnih pregleda:  12500
Broj negitivnih pregleda:  12500


Ovu funkciju ćemo iskoristiti za iscrtavanje grafika tačnosti i funkcije gubitka nakon treniranja.

In [19]:
def plot_graphs(history, epoches):
    
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title('Accuracy')
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.plot(range(0, epoches), accuracy, color='red', label='training')
    plt.plot(range(0, epoches), val_accuracy, color='orange', label='validation')
    plt.xlabel('epochs')
    plt.legend(loc='best')
    
    plt.subplot(1, 2, 2)
    plt.title('Loss')
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(range(0, epoches), loss, color='red', label='training')
    plt.plot(range(0, epoches), val_loss, color='orange', label='validation')
    plt.xlabel('epochs')
    plt.legend(loc='best')
    
    plt.show()

Nadalje ćemo eksperimentisati sa različitim rekurentnim arhitekturama. 

In [20]:
epoches = 3
batch_size = 128

## Jednostavna rekurentne neuronska mreža

In [21]:
embedding_dim = 64
output_size = 128

### LSTM rekurentna neuronska mreža 

In [22]:
embedding_dim = 64
units = 32

### Stekovana LSTM mreže

In [23]:
embedding_dim = 64
units = 32

### GRU rekurentna neuronska mreža 

In [24]:
embedding_dim = 64
units = 32

### Neuronska mreža sa pritreniranim ugnježdavanjima

Isprobaćemo i varijantu mreže koja će koristiti pritrenirane ugnježdene reprezentacije. Ovakve reprezentacije se obično uče nad većim skupom podataka i mogu biti korisne za mnoge zadatke u kojima je raspoloživi skup mnogo manjeg obima. O načinima njihovog učenja biće više reči u sekciji o nenadgledanom mašinskom učenju, a ove ćemo iskoristiti `GloVe` reprezentacije. Ceo paket sa reprezentacijama reči različitih dužina (50, 100, 200 i 300) ukupne veličine 822MB se može preuzeti sa [zvanične adrese](https://nlp.stanford.edu/projects/glove/) Stanford grupe. Mi ćemo u radu koristiti reprezentacije dužine 100 koje se nalaze u datoteci `glove.6B.100d.txt` (347.1MB) koja se može preuzeti pojedinačno npr. sa [ove](https://www.kaggle.com/terenceliu4444/glove6b100dtxt) adrese. 

Prvo ćemo pročitati iz preuzete datoteke sve podržane reči i njihove vektorske reprezentacije. U pojedinačnim redovima datoteke se prvo nalazi reči, a potom 100 realnih vrednosti koje predstavljaju njenu vektorsku reprezentaciju.

In [25]:
word_embeddings = {}

Dalje, da bismo izdvojili skup za treniranje, skup za validaciju i skup za testiranje, promešaćemo indekse skupova podataka - podrazumevano prvo imamo sve pozitivne preglede uzastopno, a zatim i sve negativne.

In [ ]:
indices = np.arange(reviews.shape[0])
np.random.shuffle(indices)

Prateći brojke iz prethodnih primera, u test skup ćemo smestiti polovinu pregleda, a ostale ćemo podeliti na preglede za traniranje i validaciju.

In [ ]:
number_of_reviews = reviews.shape[0]
test_reviews = reviews.iloc[indices[number_of_reviews//2:]]
validation_size = int(0.2*number_of_reviews)
train_reviews = reviews.iloc[indices[0:number_of_reviews//2-validation_size]]
validation_reviews = reviews.iloc[indices[number_of_reviews//2-validation_size:number_of_reviews//2]]

In [ ]:
test_reviews.shape

In [ ]:
train_reviews.shape

In [ ]:
validation_reviews.shape

Obradu ćemo izvršiti korišćenjem Kerasovog `Tokenizer` tokenizatora. Njime se mogu izdvojiti pojedinačne reči tj. tokeni zadatog teksta i konstruisati vokabular željene dužine. Kao i u prethodnim primerima, iskoristićemo 10000 najfrekventnijih reči.

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer